In [1]:
def randomprime(i):
    return random_prime(2**i-1,True,2**(i-1))

In [ ]:
l = 2048

q = randomprime(l)
p = randomprime(l)

n = p * q
phi = (p-1)*(q-1)

e = ZZ.random_element(phi)
R = IntegerModRing(n)

while gcd(e, phi) != 1:
    e = ZZ.random_element(phi)

bezout = xgcd(e,phi)
d = Integer(mod(bezout[1],phi));

In [ ]:
# Public key
(n,e)

In [ ]:
# Private key
(p,q,d)

In [ ]:
import hashlib
import binascii

from sage.crypto.util import bin_to_ascii
from random import SystemRandom

nBits = 1024
k0BitsInt = 256
k0BitsFill = '0256b'
encoding = 'utf-8'

oracle1 = hashlib.sha256()
oracle2 = hashlib.sha256()

def CharsToBinary(msg):
    bits = bin(int(binascii.hexlify(msg), 16))
    return bits

def BinaryToChars(bits):
    r = bin_to_ascii(bits)
    return r
    
def paddingOAEP(msg):
  
    randBitStr = format(SystemRandom().getrandbits(k0BitsInt), k0BitsFill)
    binMsg = CharsToBinary(msg)
    
    if len(binMsg) <= (nBits-k0BitsInt):
        k1Bits = nBits - k0BitsInt - len(binMsg) 
        zeroPaddedMsg = binMsg + ('0'*k1Bits)
    
    oracle1.update(randBitStr.encode(encoding))
    x = format(int(zeroPaddedMsg, 2) ^^ int(oracle1.hexdigest(), 16), '0768b')
    oracle2.update(x.encode(encoding))
    y = format(int(randBitStr, 2) ^^ int(oracle2.hexdigest(), 16), k0BitsFill)

    return x+y

def unpad(msg):

    x = msg[0:768]
    y = msg[768:]

    oracle2.update(x.encode(encoding))
    r = format(int(y,2) ^^ int(oracle2.hexdigest(), 16), k0BitsFill)

    oracle1.update(r.encode(encoding))
    msgWith0s = format(int(x,2) ^^ int(oracle1.hexdigest(), 16), '0768b')

    msg = BinaryToChars(msgWith0s)
    return msg

In [ ]:
def encrypt(msg):
    msgWithPad = paddingOAEP(msg)
    m = '0b' + msgWithPad
    zz = ZZ(m)
    a = R(zz)
    ct = a**e
    return ct

def decpryt(ct):
    b = R(ct)
    dm = b**d
    zz = ZZ(dm)
    inteiro = Integer(zz)
    m1 = inteiro.binary()
    m = '00'+ m1
    msg = unpad(m)
    return msg

In [ ]:
from random import randint

msg = "Hello world"

ct = encrypt(msg)
m = decpryt(ct)

print(msg == m)